<a href="https://colab.research.google.com/github/pabrey/datasciencecoursera/blob/master/4c_transfor_out_sim_to_input_miero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import io
import matplotlib.pyplot as plt
import numpy as np
import json
from google.colab import files
import time
from datetime import datetime
import math

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# create functions for analysys

## function process data

In [0]:
def load_data(location):
  Separator = ';'
  df =  pd.read_csv(location, sep=Separator)
  df.columns = ["CASE_ID", "ACTIVITY",	"DATE"] 
  df_order = df.sort_values(['CASE_ID', 'DATE'])
  df_order["TIMESTAMP_IN"] = (df_order['DATE']).apply(lambda x:  time.mktime( datetime.strptime(str(x.replace(' UTC','')), '%d/%m/%Y %H:%M:%S').timetuple()))
  return df_order

create difference timesatamps

In [0]:

def process_times(df_input):

  def processDepartureTime(nodes):

    temp_time = 0
    len_nodes = len(nodes)
    sal = []
    mi_index = 0
    for index, node in nodes.iteritems():
      mi_index = mi_index +1

      if (mi_index == len_nodes):
        sal.append(node)
      else:
        sal.append((nodes[index+1]))

    return sal
  data_time = df_input.sort_values(["CASE_ID", "TIMESTAMP_IN"]).reset_index()
  data_time["TIMESTAMP_OUT"] = data_time.groupby("CASE_ID")["TIMESTAMP_IN"].transform(lambda x: processDepartureTime(x))
  data_time['DIFFERENCE'] = data_time['TIMESTAMP_OUT'] - data_time['TIMESTAMP_IN']
  data_time = data_time.sort_values(["CASE_ID", "TIMESTAMP_IN"])
  return data_time

show data

In [0]:
def calculate_average_time(input_df):
  def average_time(nodes):
      min_date = None
      max_date = None

      for index, node in nodes.iterrows():      
          temp_date = node["TIMESTAMP_IN"] 
          if ((min_date == None) or min_date>temp_date):
            min_date = temp_date
          if ((max_date == None) or max_date<temp_date):
            max_date = temp_date

      return max_date- min_date

  data_average = input_df.sort_values(["CASE_ID", "TIMESTAMP_IN"]).groupby("CASE_ID")["CASE_ID", "TIMESTAMP_IN", "DIFFERENCE"].apply(lambda x : average_time(x))
  data_average = data_average.sort_values(["CASE_ID", "TIMESTAMP_IN"])
  return data_average


def show_results(input_df):
  print("Mean time per nod")
  print(input_df.groupby("ACTIVITY")["DIFFERENCE"].mean())
  data_average = calculate_average_time(input_df)
  print("Mean time per user")
  print(data_average.head(10))
  print("Mean time of all users")
  print(data_average.mean())


# Data for sinulation_00

In [53]:
Location = 'drive/My Drive/data/prosae/testAllRoutesMean/input_data_miner_mean_00.csv'
df_order = load_data(Location)
df_times = process_times(df_order)
df_times.head()

,index,CASE_ID,ACTIVITY,DATE,TIMESTAMP_IN,TIMESTAMP_OUT,DIFFERENCE
0,0,2,admision,12/02/1975 22:47:09 UTC,161477229.0,161477998.0,769.0
1,1,2,triaje,12/02/1975 22:59:58 UTC,161477998.0,161483768.0,5770.0
2,2,2,rx solicitud,13/02/1975 00:36:08 UTC,161483768.0,161523340.0,39572.0
3,3,2,Aalta,13/02/1975 11:35:40 UTC,161523340.0,161523340.0,0.0
4,4,4,admision,12/02/1975 22:59:58 UTC,161477998.0,161478572.0,574.0


In [54]:
show_results(df_times)

Mean time per nod
ACTIVITY
Aalta                 0.000000
admision            742.846442
interconsulta     20182.769231
respuesta         20818.800000
rx informe        12762.142857
rx solicitud      12611.047619
triaje           329974.530612
Name: DIFFERENCE, dtype: float64


TypeError: ignored

In [49]:
len(df_times)

17808

In [55]:
df_times[df_times["ACTIVITY"] == "admision"].head()

,index,CASE_ID,ACTIVITY,DATE,TIMESTAMP_IN,TIMESTAMP_OUT,DIFFERENCE
0,0,2,admision,12/02/1975 22:47:09 UTC,161477229.0,161477998.0,769.0
4,4,4,admision,12/02/1975 22:59:58 UTC,161477998.0,161478572.0,574.0
7,7,5,admision,12/02/1975 23:09:32 UTC,161478572.0,161480436.0,1864.0
11,11,6,admision,12/02/1975 23:40:36 UTC,161480436.0,161481901.0,1465.0
15,15,9,admision,13/02/1975 00:05:01 UTC,161481901.0,161482101.0,200.0


# Data for sinulation_00 extended time

In [47]:
Location = 'drive/My Drive/data/prosae/testAllRoutesMean/input_data_miner_mean_00_extended_time.csv'
df_order = load_data(Location)
df_times = process_times(df_order)
df_times.head()

,index,CASE_ID,ACTIVITY,DATE,TIMESTAMP_IN,TIMESTAMP_OUT,DIFFERENCE
0,0,1,admision,12/02/1975 22:33:10 UTC,161476390.0,161477229.0,839.0
1,1,1,triaje,12/02/1975 22:47:09 UTC,161477229.0,161481639.0,4410.0
2,2,1,rx solicitud,13/02/1975 00:00:39 UTC,161481639.0,161483091.0,1452.0
3,3,1,rx exploracion,13/02/1975 00:24:51 UTC,161483091.0,162733984.0,1250893.0
4,4,1,rx informe,27/02/1975 11:53:04 UTC,162733984.0,162735018.0,1034.0


In [48]:
show_results(df_times)

Mean time per nod
ACTIVITY
Aalta             0.000000e+00
admision          7.479814e+02
interconsulta     1.256853e+05
respuesta         4.025176e+04
rx exploracion    3.103546e+06
rx informe        2.235289e+04
rx solicitud      3.251732e+04
triaje            2.790691e+06
Name: DIFFERENCE, dtype: float64


TypeError: ignored

# Create file for minero directly

In [0]:
Location = 'drive/My Drive/data/prosae/testAllRoutesMean/res_simulacion_means.csv'


Separator = ';'
df =  pd.read_csv(Location, sep=Separator)

df.columns = ["CASE_ID", "ACTIVITY",	"TIMESTAMP_IN",	"TIMESTAMP_OUT", "COLOR"] 
df_order = df.sort_values(['CASE_ID', 'TIMESTAMP_IN'])

df_out = df.copy().sort_values(["CASE_ID", "TIMESTAMP_IN"])

id_filter = df_out[df_out["TIMESTAMP_IN"] == 0]["CASE_ID"]
df_filter_out = df_out[~df_out["CASE_ID"].isin(id_filter)]
id_filter = df_filter_out[df_filter_out["TIMESTAMP_OUT"] == 0]["CASE_ID"]
df_filter_out = df_filter_out[~df_filter_out["CASE_ID"].isin(id_filter)]

df_filter_out["DIFFERENCE"] = df_filter_out["TIMESTAMP_OUT"]-df_filter_out["TIMESTAMP_IN"]
df_filter_out["DATE_IN"] = (pd.to_datetime(df_filter_out["TIMESTAMP_IN"]*1000,unit='ms'))


df_sal = df_filter_out[["CASE_ID", "ACTIVITY", "DATE_IN"]]
df_sal["TIMESTAMP"] = df_sal["DATE_IN"].dt.strftime('%d/%m/%Y %H:%M:%S')+ " UTC"
df_sal = df_sal[["CASE_ID", "ACTIVITY", "TIMESTAMP"]].reset_index()


df_sal[["CASE_ID","ACTIVITY","TIMESTAMP"]].sort_values(["CASE_ID", "TIMESTAMP"]).to_csv(r'input_data_miner_means.csv', index=False, sep = ';')
files.download('input_data_miner_means.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
